In [1]:
import torch as t
import torch.nn as nn
from pathlib import Path
from DiagnosisAI.models.AutoEncoder_BRAIN import Segmenter
import pickle
import numpy as np
import nibabel as nib
from sklearn.model_selection import train_test_split
import pytorch_lightning as pl
import matplotlib.pyplot as plt

/home/michalheit/miniconda3/envs/mgr_dp/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dir_path = Path("../datasets/brain/train_images_max_area")
cases = []
for dirfile in dir_path.iterdir():
    for file in dirfile.iterdir():
        cases.append(pickle.load(open(file, 'rb')))

In [48]:
imgs = [[elem['flair'], elem['seg']] for elem in cases]

In [49]:
imgs_t = [t.tensor(elem) for elem in imgs]

In [50]:
imgs_train, imgs_test = train_test_split(imgs_t, test_size=0.2, random_state=42)

In [51]:
imgs_train, imgs_val = train_test_split(imgs_train, test_size=0.1, random_state=42)

In [52]:
train_loader = t.utils.data.DataLoader(imgs_train, batch_size=8, num_workers=2)
val_loader = t.utils.data.DataLoader(imgs_val, batch_size=8, num_workers=2)

In [55]:
img, label = next(iter(train_loader))

ValueError: too many values to unpack (expected 2)

In [54]:
img[7]

tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]], dtype=torch.float64)

In [22]:
segmenter = Segmenter()
model_checkpoint = pl.callbacks.ModelCheckpoint(dirpath='/content/checkpoints')
early_stopping = pl.callbacks.EarlyStopping(monitor='val_loss', patience=2)

trainer = pl.Trainer(callbacks=[model_checkpoint, early_stopping], gpus=1, max_epochs=100)
trainer.fit(segmenter, train_dataloaders=train_loader, val_dataloaders=val_loader)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Missing logger folder: /mnt/e/mgr/examples/lightning_logs

  | Name          | Type             | Params
---------------------------------------------------
0 | network       | Unet             | 14.3 M
1 | loss_function | CrossEntropyLoss | 0     
2 | accuracy      | Accuracy         | 0     
---------------------------------------------------
14.3 M    Trainable params
0         Non-trainable params
14.3 M    Total params
57.312    Total estimated model params size (MB)


Validation sanity check:   0%|          | 0/2 [00:00<?, ?it/s]

/home/michalheit/miniconda3/envs/mgr_dp/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


ValueError: too many values to unpack (expected 2)

In [12]:
# value 1 - core
# value 2 - invaded
# value 4 - enhanced